In [1]:
from google.colab import drive
drive.mount("/content/drive")

import warnings
warnings.simplefilter(action='ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('/content/drive/MyDrive/KB_AI')

df = pd.read_csv('./data/bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [3]:
df_train =  df.iloc[:,:-1]
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown


### 데이터 확인

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
dtypes: int64(7), object(9)
memory usage: 1.4+ MB


In [5]:
#결측치 확인
for col in df_train.columns:
  print('{:>10}\t : {}'.format(col, df_train[col].isnull().sum()))

       age	 : 0
       job	 : 0
   marital	 : 0
 education	 : 0
   default	 : 0
   balance	 : 0
   housing	 : 0
      loan	 : 0
   contact	 : 0
       day	 : 0
     month	 : 0
  duration	 : 0
  campaign	 : 0
     pdays	 : 0
  previous	 : 0
  poutcome	 : 0


In [6]:
#연속형 변수

numerical = df_train.dtypes[df_train.dtypes!='object'].index
numerical_cols = df_train[numerical].columns
print(len(numerical_cols))
print(numerical_cols)

7
Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')


In [7]:
# 범주형 변수

categorical = df_train.dtypes[df_train.dtypes=='object'].index
categorical_cols = df_train[categorical].columns
print(len(categorical_cols))
print(categorical_cols)

9
Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')


In [8]:
# 이상치 제거 : IQR
from collections import Counter

def detect_outliers(df, n, features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1

        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(k for k, v in outlier_indices.items() if v > n)

    return multiple_outliers

Outliers_to_drop = detect_outliers(df_train, 2, numerical_cols)

In [9]:
df_train.loc[Outliers_to_drop]

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
2665,79,retired,married,secondary,no,723,no,no,telephone,9,sep,276,1,79,3,failure
2778,76,housemaid,divorced,primary,no,1411,no,no,telephone,19,oct,233,2,187,3,success
2809,82,retired,married,primary,no,8603,no,no,cellular,22,oct,306,3,107,1,success
2820,77,management,married,unknown,no,1780,yes,no,cellular,23,oct,221,2,183,3,success
2931,76,retired,married,primary,no,3324,no,no,cellular,13,nov,122,1,92,3,success
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,57,services,married,secondary,no,0,yes,no,cellular,14,may,16,9,282,3,other
10157,44,management,married,tertiary,no,1451,yes,no,cellular,5,feb,80,7,80,15,failure
10573,44,admin.,single,secondary,no,985,yes,yes,cellular,11,may,9,7,325,19,failure
10765,37,management,married,tertiary,no,26,no,no,cellular,17,jun,151,7,62,4,failure


In [10]:
#220개 제거

df_train = df_train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10942 entries, 0 to 10941
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        10942 non-null  int64 
 1   job        10942 non-null  object
 2   marital    10942 non-null  object
 3   education  10942 non-null  object
 4   default    10942 non-null  object
 5   balance    10942 non-null  int64 
 6   housing    10942 non-null  object
 7   loan       10942 non-null  object
 8   contact    10942 non-null  object
 9   day        10942 non-null  int64 
 10  month      10942 non-null  object
 11  duration   10942 non-null  int64 
 12  campaign   10942 non-null  int64 
 13  pdays      10942 non-null  int64 
 14  previous   10942 non-null  int64 
 15  poutcome   10942 non-null  object
dtypes: int64(7), object(9)
memory usage: 1.3+ MB


In [11]:
'''Skewness(비대칭도)란 분포가 얼마나 비대칭을 띄는가 알려주는 척도입니다.
(비대칭도: a=0이면 정규분포, a<0 이면 오른쪽으로 치우침, a>0이면 왼쪽으로 치우침)
Kurtosis (첨도)란 확률분포의 뾰족한 정도를 나타내는 척도입니다.
관측치들이 어느 정도 집중적으로 중심에 몰려 있는가를 측정할 때 사용됩니다.'''

for col in numerical_cols :
  print('{:15}'.format(col), 'Skewness : {:05.2f}'.format(df_train[col].skew()), '    ', 'Kurtosis:{:05.2f}'.format(df_train[col].kurt()))

age             Skewness : 00.82      Kurtosis:00.51
balance         Skewness : 07.10      Kurtosis:90.67
day             Skewness : 00.11      Kurtosis:-1.07
duration        Skewness : 02.18      Kurtosis:07.70
campaign        Skewness : 05.60      Kurtosis:58.09
pdays           Skewness : 02.56      Kurtosis:07.54
previous        Skewness : 08.01      Kurtosis:136.49


### 데이터 전처리


#### 연령별 퇴직연금제도 가입률 활용
영업점에서 퇴직연금 상품 제안을 경험해보면서, 퇴직연금 가입에 가장 영향을 많이 미치는 요소는 연령대라고 느낌 -> 이에 따라 KOSIS의 '2015~2021 성별 연령별 퇴직연금제도 가입률' 데이터를 활용하여, 연령대에 따른 평균 가입률만큼 랜덤하게 퇴직연금 가입여부 데이터를 생성

In [12]:
#각 년도의 .2 번째가 총 가입률을 의미한다

age_ratio = pd.read_csv('./data/성별_연령별_퇴직연금제도_가입률.csv', encoding='cp949')
age_ratio = age_ratio.iloc[1:]
age_ratio.head()

,연령별(1),2015,2015.1,2015.2,2015.3,2015.4,2015.5,2015.6,2015.7,2015.8,...,2020.8,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8
1,연령별(1),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%),...,가입률 (%),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%),가입 대상 근로자 수 (명),가입 근로자 수 (명),가입률 (%)
2,계,10336230,4977407,48.2,6305675,3168336,50.2,4117056,1809071,44.9,...,51.5,11956524,6367827,53.3,7034691,3790163,53.9,4921833,2577664,52.4
3,20세 미만,31468,7273,23.1,14682,3269,22.3,12518,4004,23.9,...,22.5,17645,4007,22.7,9855,2026,20.6,7790,1981,25.4
4,20 - 24세,346958,127939,36.9,124313,36855,29.6,207413,91084,40.9,...,43.8,292172,117350,40.2,121032,42685,35.3,171140,74665,43.6
5,25 - 29세,1021778,521956,51.1,508959,254962,50.1,501199,266994,52.1,...,56.3,1103887,592129,53.6,566540,285446,50.4,537347,306683,57.1


In [13]:
#각 년도마다 총 9개의 열을 가지고 있고, 그 중에서 3번째 열이 각 나이대 별 모든 성별의 퇴직연금 가입률

year = 2015
df_age_ratio = pd.DataFrame(age_ratio.iloc[:,0])
for n in range(3,len(age_ratio.columns),9):
  df_age_ratio[year] = age_ratio.iloc[:, n]
  year+=1

In [14]:
df_age_ratio.head()

,연령별(1),2015,2016,2017,2018,2019,2020,2021
1,연령별(1),가입률 (%),가입률 (%),가입률 (%),가입률 (%),가입률 (%),가입률 (%),가입률 (%)
2,계,48.2,49.3,50.2,51.3,51.5,52.4,53.3
3,20세 미만,23.1,25.4,24.7,20.8,15.9,21.2,22.7
4,20 - 24세,36.9,38.1,37.9,36.7,35.9,39.8,40.2
5,25 - 29세,51.1,51.4,51.9,51.5,51.5,52.8,53.6


In [15]:
#각 열의 3번째 행은 그해 년도의 총 퇴직연금 가입률이므로 사용할 필요가 없다고 판단하여 제외
df_age_ratio.rename(columns={'연령별(1)':'나이대'}, inplace=True)
df_age_ratio.drop([1,2],axis=0,inplace=True)
df_age_ratio.set_index(keys='나이대', drop=True, inplace=True)

In [16]:
df_age_ratio = df_age_ratio.astype(float)
df_age_ratio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 20세 미만 to 65세 이상
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2015    11 non-null     float64
 1   2016    11 non-null     float64
 2   2017    11 non-null     float64
 3   2018    11 non-null     float64
 4   2019    11 non-null     float64
 5   2020    11 non-null     float64
 6   2021    11 non-null     float64
dtypes: float64(7)
memory usage: 704.0+ bytes


In [17]:
df_age_ratio['avg'] = df_age_ratio.mean(axis=1)
df_age_ratio

,2015,2016,2017,2018,2019,2020,2021,avg
나이대,,,,,,,,
20세 미만,23.1,25.4,24.7,20.8,15.9,21.2,22.7,21.971429
20 - 24세,36.9,38.1,37.9,36.7,35.9,39.8,40.2,37.928571
25 - 29세,51.1,51.4,51.9,51.5,51.5,52.8,53.6,51.971429
30 - 34세,57.1,58.2,59.4,60.2,59.7,59.8,60.6,59.285714
35 - 39세,55.0,56.4,58.0,59.6,59.8,60.4,61.4,58.657143
40 - 44세,50.8,52.5,54.3,56.3,57.2,57.9,59.1,55.442857
45 - 49세,49.2,50.4,51.6,53.2,53.7,54.5,55.6,52.600000
50 - 54세,46.9,48.5,49.8,51.4,52.0,52.9,53.7,50.742857
55 - 59세,40.8,42.7,44.3,47.0,48.1,49.5,50.9,46.185714


#### 산업대 분류별 퇴직연금제도 가입률 활용

데이터의 타당성을 부여하기 위해, 위와 동일한 KOSIS에서 제공하는 '2015~2021 산업대 분류별 퇴직연금제도 가입률' 데이터를 활용하여, 연령대에 따른 평균 가입률만큼 랜덤하게 퇴직연금 가입여부 데이터를 생성

In [18]:
#각 년도의 .2 번째가 총 가입률을 의미한다

job_ratio = pd.read_csv('./data/산업대분류별_퇴직연금제도_가입률.csv', encoding='cp949')
job_ratio = job_ratio.iloc[1:]
job_ratio.head()

,산업분류별,2015,2015.1,2015.2,2016,2016.1,2016.2,2017,2017.1,2017.2,...,2018.2,2019,2019.1,2019.2,2020,2020.1,2020.2,2021,2021.1,2021.2
1,합계,10336230,4977407,48.2,10588453,5221202,49.3,10830161,5437938,50.2,...,51.3,11508858,5929473,51.5,11865340,6219497,52.4,11956524,6367827,53.3
2,"농업, 임업 및 어업",33430,8527,25.5,35826,8960,25,38152,9735,25.5,...,23.9,43628,10041,23,45600,10046,22,45860,10175,22.2
3,광업,10570,6509,61.6,10522,6573,62.5,9587,5740,59.9,...,64.1,10597,6822,64.4,10256,6675,65.1,9767,6163,63.1
4,제조업,3323506,1980687,59.6,3334850,2000473,60,3332020,2043477,61.3,...,62.2,3424109,2134158,62.3,3463491,2165134,62.5,3447154,2190738,63.6
5,"전기, 가스, 증기 및 공기조절 공급업",63643,40769,64.1,74316,52112,70.1,76783,54601,71.1,...,68.1,68887,45461,66,73486,49215,67,74918,50826,67.8


In [19]:
#각 년도마다 총 3개의 열을 가지고 있고, 그 중에서 3번째 열이 각 나이대 별 모든 성별의 퇴직연금 가입률

year = 2015
df_job_ratio = pd.DataFrame(job_ratio.iloc[:,0])
for n in range(3,len(job_ratio.columns),3):
  df_job_ratio[year] = job_ratio.iloc[:, n]
  year+=1

In [20]:
df_job_ratio.head()

,산업분류별,2015,2016,2017,2018,2019,2020,2021
1,합계,48.2,49.3,50.2,51.3,51.5,52.4,53.3
2,"농업, 임업 및 어업",25.5,25,25.5,23.9,23,22,22.2
3,광업,61.6,62.5,59.9,64.1,64.4,65.1,63.1
4,제조업,59.6,60,61.3,62.2,62.3,62.5,63.6
5,"전기, 가스, 증기 및 공기조절 공급업",64.1,70.1,71.1,68.1,66,67,67.8


In [21]:
#합계 행은 총 퇴직연금 가입률이므로 사용할 필요가 없다고 판단하여 제외

df_job_ratio.drop(1,axis=0,inplace=True)
df_job_ratio.set_index(keys='산업분류별', drop=True, inplace=True)
df_job_ratio.head()

,2015,2016,2017,2018,2019,2020,2021
산업분류별,,,,,,,
"농업, 임업 및 어업",25.5,25,25.5,23.9,23,22,22.2
광업,61.6,62.5,59.9,64.1,64.4,65.1,63.1
제조업,59.6,60,61.3,62.2,62.3,62.5,63.6
"전기, 가스, 증기 및 공기조절 공급업",64.1,70.1,71.1,68.1,66,67,67.8
"수도, 하수 및 폐기물 처리, 원료 재생업",51.5,52.7,53.1,55,57.7,58.7,60.2


In [22]:
df_job_ratio = df_job_ratio.astype(float)
df_job_ratio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 농업, 임업 및 어업 to 국제 및 외국기관
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2015    21 non-null     float64
 1   2016    21 non-null     float64
 2   2017    21 non-null     float64
 3   2018    21 non-null     float64
 4   2019    21 non-null     float64
 5   2020    21 non-null     float64
 6   2021    21 non-null     float64
dtypes: float64(7)
memory usage: 1.3+ KB


In [40]:
df_job_ratio['avg'] = df_job_ratio.mean(axis=1)
df_job_ratio

,2015,2016,2017,2018,2019,2020,2021,avg
산업분류별,,,,,,,,
"농업, 임업 및 어업",25.5,25.0,25.5,23.9,23.0,22.0,22.2,23.871429
광업,61.6,62.5,59.9,64.1,64.4,65.1,63.1,62.957143
제조업,59.6,60.0,61.3,62.2,62.3,62.5,63.6,61.642857
"전기, 가스, 증기 및 공기조절 공급업",64.1,70.1,71.1,68.1,66.0,67.0,67.8,67.742857
"수도, 하수 및 폐기물 처리, 원료 재생업",51.5,52.7,53.1,55.0,57.7,58.7,60.2,55.557143
건설업,32.9,33.5,33.3,33.5,33.5,33.4,33.2,33.328571
도매 및 소매업,42.2,43.0,43.4,43.3,43.5,43.7,43.3,43.200000
운수 및 창고업,39.4,40.7,41.4,43.5,42.6,47.2,49.2,43.428571
숙박 및 음식점업,22.8,23.0,25.1,23.9,24.8,23.5,23.9,23.857143


In [41]:
#KOSIS 분류에 없는 kaggle jobs : retired, unemployed, housemaid, unknown, student

kaggle_jobs = df_train['job'].unique()
kaggle_jobs

array(['admin.', 'technician', 'services', 'management', 'retired',
       'blue-collar', 'unemployed', 'entrepreneur', 'housemaid',
       'unknown', 'self-employed', 'student'], dtype=object)

In [72]:
kaggle_jobs = ['admin.', 'technician', 'services', 'management',
       'blue-collar', 'entrepreneur', 'self-employed']

In [73]:
df_job_ratio_new = pd.DataFrame()
df_job_ratio_new.index = kaggle_jobs
df_job_ratio_new['avg'] = 0
df_job_ratio_new

,avg
admin.,0
technician,0
services,0
management,0
blue-collar,0
entrepreneur,0
self-employed,0


In [74]:
df_job_ratio['avg']

산업분류별
농업, 임업 및 어업                        23.871429
광업                                 62.957143
제조업                                61.642857
전기, 가스, 증기 및 공기조절 공급업              67.742857
수도, 하수 및 폐기물 처리, 원료 재생업            55.557143
건설업                                33.328571
도매 및 소매업                           43.200000
운수 및 창고업                           43.428571
숙박 및 음식점업                          23.857143
정보통신업                              62.485714
금융 및 보험업                           68.671429
부동산업                               25.457143
전문, 과학 및 기술 서비스업                   49.557143
사업시설 관리 및 사업 지원 및 임대 서비스업          54.728571
공공행정, 국방 및 사회보장 행정                 22.614286
교육 서비스업                            36.328571
보건업 및 사회복지 서비스업                    56.128571
예술, 스포츠 및 여가관련 서비스업                44.657143
협회 및 단체, 수리  및 기타 개인 서비스업          36.185714
가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동    38.314286
국제 및 외국기관                           0.514286
Name: avg, dtype: float64

In [75]:
#admin : 공공행정, 국방 및 사회보장 행정
df_job_ratio_new['avg'][0] = df_job_ratio['avg']['공공행정, 국방 및 사회보장 행정']

#technician : 전문, 과학 및 기술 서비스업/정보통신업
df_job_ratio_new['avg'][1] = (df_job_ratio['avg']['전문, 과학 및 기술 서비스업']
                              +df_job_ratio['avg']['정보통신업'])/2

#services
#협회 및 단체, 수리 및 기타 개인 서비스업/예술, 스포츠 및 여가관련 서비스업/보건업 및 사회복지 서비스업
#교육 서비스업
df_job_ratio_new['avg'][2] = (df_job_ratio['avg']['협회 및 단체, 수리  및 기타 개인 서비스업']
                              +df_job_ratio['avg']['예술, 스포츠 및 여가관련 서비스업']
                              +df_job_ratio['avg']['보건업 및 사회복지 서비스업']
                              +df_job_ratio['avg']['교육 서비스업'])/4

#management : 국제 및 외국기관/금융 및 보험업
df_job_ratio_new['avg'][3] = (df_job_ratio['avg']['국제 및 외국기관']
                              +df_job_ratio['avg']['금융 및 보험업'])/2

#blue-collar(생산&서비스업 종사자, 육체 노동 종사자)
#농업, 임업 및 어업/광업/제조업/전기, 가스, 증기 및 공기조절 공급업/수도, 하수 및 폐기물 처리, 원료 재생업/건설업
#도매 및 소매업/운수 및 창고업
df_job_ratio_new['avg'][4] = (df_job_ratio['avg']['농업, 임업 및 어업']
                              +df_job_ratio['avg']['광업']
                              +df_job_ratio['avg']['제조업']
                              +df_job_ratio['avg']['전기, 가스, 증기 및 공기조절 공급업']
                              +df_job_ratio['avg']['수도, 하수 및 폐기물 처리, 원료 재생업']
                              +df_job_ratio['avg']['건설업']
                              +df_job_ratio['avg']['도매 및 소매업']
                              +df_job_ratio['avg']['운수 및 창고업'])/8

#entrepreneur : 사업시설 관리 및 사업 지원 및 임대 서비스업/부동산업/숙박 및 음식점업
df_job_ratio_new['avg'][5] = (df_job_ratio['avg']['사업시설 관리 및 사업 지원 및 임대 서비스업']
                              +df_job_ratio['avg']['부동산업']
                              +df_job_ratio['avg']['숙박 및 음식점업'])/2

#self-employed : 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동
df_job_ratio_new['avg'][6] = df_job_ratio['avg']['가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동']

In [76]:
df_job_ratio_new

,avg
admin.,22.614286
technician,56.021429
services,43.325000
management,34.592857
blue-collar,48.966071
entrepreneur,52.021429
self-employed,38.314286


#### 연령별&산업대분류별 통합 표

In [77]:
df_ratio = pd.DataFrame()
df_ratio.index = np.array(df_job_ratio_new.index)
for col in df_age_ratio.index:
  df_ratio[col] = 0

df_ratio

,20세 미만,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65세 이상
admin.,0,0,0,0,0,0,0,0,0,0,0
technician,0,0,0,0,0,0,0,0,0,0,0
services,0,0,0,0,0,0,0,0,0,0,0
management,0,0,0,0,0,0,0,0,0,0,0
blue-collar,0,0,0,0,0,0,0,0,0,0,0
entrepreneur,0,0,0,0,0,0,0,0,0,0,0
self-employed,0,0,0,0,0,0,0,0,0,0,0


In [78]:
df_age_ratio.index, df_age_ratio.columns

(Index(['20세 미만', '20 - 24세', '25 - 29세', '30 - 34세', '35 - 39세', '40 - 44세',
        '45 - 49세', '50 - 54세', '55 - 59세', '60 - 64세', '65세 이상'],
       dtype='object', name='나이대'),
 Index([2015, 2016, 2017, 2018, 2019, 2020, 2021, 'avg'], dtype='object'))

In [79]:
df_job_ratio_new.index, df_job_ratio_new.columns

(Index(['admin.', 'technician', 'services', 'management', 'blue-collar',
        'entrepreneur', 'self-employed'],
       dtype='object'),
 Index(['avg'], dtype='object'))

In [80]:
for age_col in df_age_ratio.columns:
  for age_idx in df_age_ratio.index:
      for job_idx in df_job_ratio_new.index:
        df_ratio[age_idx][job_idx] = df_age_ratio[age_col][age_idx]*df_job_ratio_new['avg'][job_idx]*0.0001



In [81]:
df_ratio

,20세 미만,20 - 24세,25 - 29세,30 - 34세,35 - 39세,40 - 44세,45 - 49세,50 - 54세,55 - 59세,60 - 64세,65세 이상
admin.,0.049687,0.085773,0.117530,0.134070,0.132649,0.125380,0.118951,0.114751,0.104446,0.082768,0.063708
technician,0.123087,0.212481,0.291151,0.332127,0.328606,0.310599,0.294673,0.284269,0.258739,0.205038,0.157820
services,0.095191,0.164326,0.225166,0.256855,0.254132,0.240206,0.227890,0.219843,0.200100,0.158570,0.122053
management,0.076005,0.131206,0.179784,0.205086,0.202912,0.191793,0.181958,0.175534,0.159770,0.126610,0.097453
blue-collar,0.107585,0.185721,0.254484,0.290299,0.287221,0.271482,0.257562,0.248468,0.226153,0.179216,0.137944
entrepreneur,0.114299,0.197310,0.270363,0.308413,0.305143,0.288422,0.273633,0.263972,0.240265,0.190398,0.146552
self-employed,0.084182,0.145321,0.199125,0.227149,0.224741,0.212425,0.201533,0.194418,0.176957,0.140230,0.107937


### 학습용 데이터 가공

퇴직연금 가입 여부인 'pension' 컬럼을 추가

In [82]:
df_train['pension'] = 0

In [83]:
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,pension
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,0
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,0
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,0
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,0
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,0


In [95]:
for i in range(len(df_train)):
  if df_train['job'][i] in df_ratio.index:
    if df_train['age'][i] < 20 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[0]][df_train['job'][i]])
    elif df_train['age'][i] <= 24 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[1]][df_train['job'][i]])
    elif df_train['age'][i] <= 29 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[2]][df_train['job'][i]])
    elif df_train['age'][i] <= 34 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[3]][df_train['job'][i]])
    elif df_train['age'][i] <= 39 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[4]][df_train['job'][i]])
    elif df_train['age'][i] <= 44 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[5]][df_train['job'][i]])
    elif df_train['age'][i] <= 49 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[6]][df_train['job'][i]])
    elif df_train['age'][i] <= 54 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[7]][df_train['job'][i]])
    elif df_train['age'][i] <= 59 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[9]][df_train['job'][i]])
    elif df_train['age'][i] <= 64 :
      df_train['pension'][i] = np.random.binomial(n=1, p=df_ratio[df_age_ratio.index[10]][df_train['job'][i]])
  else:
    df_train['pension'][i] = 0


In [96]:
df_train['pension'].sum()

1980

In [88]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10942 entries, 0 to 10941
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        10942 non-null  int64 
 1   job        10942 non-null  object
 2   marital    10942 non-null  object
 3   education  10942 non-null  object
 4   default    10942 non-null  object
 5   balance    10942 non-null  int64 
 6   housing    10942 non-null  object
 7   loan       10942 non-null  object
 8   contact    10942 non-null  object
 9   day        10942 non-null  int64 
 10  month      10942 non-null  object
 11  duration   10942 non-null  int64 
 12  campaign   10942 non-null  int64 
 13  pdays      10942 non-null  int64 
 14  previous   10942 non-null  int64 
 15  poutcome   10942 non-null  object
 16  pension    10942 non-null  int64 
dtypes: int64(8), object(9)
memory usage: 1.4+ MB


In [97]:
df_train.to_csv('./data/randomly_preprocessed_by_age_and_job.csv', encoding='utf-8')